In [1]:
import os

from langchain.chains import LLMChain, SimpleSequentialChain, SequentialChain
from langchain.llms import HuggingFaceHub
from langchain.prompts import ChatPromptTemplate
from dotenv import load_dotenv

load_dotenv()
# Initialize LLaMA models from HuggingFaceHub (replace with actual repo ID and API key)
llama_3 = HuggingFaceHub(repo_id="meta-llama/Meta-Llama-3-8B-Instruct")
#llama_2 = HuggingFaceHub(repo_id="meta-llama/Meta-Llama-3-8B-Instruct")


/var/folders/v5/3nrys0b97rg31_kvjlgmc7w40000gn/T/ipykernel_966/2884609304.py:9: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEndpoint``.
  llama_3 = HuggingFaceHub(repo_id="meta-llama/Meta-Llama-3-8B-Instruct")


In [2]:
from langchain import PromptTemplate

def prompt_builder(model_type="llama-3", system_prompt="", context="", prompt_data=""):

    if "llama-2" in model_type:
        template = """<s>[INST] <<SYS>> {system_prompt} <</SYS>> {context} {prompt_data}  [/INST] """
    elif "llama-3" in model_type:
        template = """<|begin_of_text|><|start_header_id|>system<|end_header_id|> {system_prompt} <|eot_id|>
<|start_header_id|>user<|end_header_id|> {context} {prompt_data} <|eot_id|>
<|start_header_id|>assistant<|end_header_id|> """
    elif "mixtral" in model_type:
        template = """<s> [INST] {system_prompt} {context} {prompt_data} [/INST] """
    else:
        raise NotImplementedError(f"Can not work for {model_type=}")

    full_template = template.format(system_prompt=system_prompt, context=context, prompt_data=prompt_data)
    prompt = PromptTemplate(
        template=full_template,)
    return prompt

In [3]:
import pandas as pd
df = pd.read_csv("amazon.csv")

In [4]:
discount_percentage = df.discount_percentage
rating = df.rating

In [5]:
list(rating[:15])

['4.2',
 '4.0',
 '3.9',
 '4.2',
 '4.2',
 '3.9',
 '4.1',
 '4.3',
 '4.2',
 '4.0',
 '4.3',
 '4.2',
 '4.4',
 '4.2',
 '4.1']

In [6]:
# Define prompt templates for both models
system_prompt_template_1 = "You are a online store assistant. Classifying if an item is [Good, Fair, Bad, Very Bad] in <classify></classify> tag."
template_1 = """
If an item get bad review on a discount, it's a bad item.
Here are some examples:
1. discount percentage: 62% and rating: 2.2 is "Very Bad".
2. discount percentage: 30% and rating: 2.2 is "Bad".
3. discount percentage: 30% and rating 4 is "Fair".
4. discount percentage: 10% and rating: 4.5 is "Good".
Now given discount percentage: {discount_percentage} and rating: {rating}, give only a 1 word review in [Good, Fair, Bad, Very Bad].
"""
# prompt_1 = ChatPromptTemplate.from_template(template_1)
prompt_1 = prompt_builder(system_prompt=system_prompt_template_1, prompt_data=template_1)
print(type(prompt_1))
chain_1 = LLMChain(llm=llama_3, prompt=prompt_1, output_key="Review_status")


system_prompt_template_2 = "You are a helpful text assistant. Get the value within the <classify></classify> tags and output"
template_2= """
Here are some example:
1. Random descrption that is not related to the task <classify>"Very Bad"</classify> and you just get the value in the tags:"Very Bad".
2. <|begin_of_text|><|start_header_id|>system<|end_header_id|> You are a online store sales assistant. Given the item is claissified as Good or Bad, decide if we will keep selling them as [Yes, Neutral, No]. <|eot_id|>\n<|start_header_id|> <classify>"Bad"</classify>:"Bad"
3. if we will keep selling them as [Yes, Neutral, No]. <|eot_id|>\n<|start_header_id|>user<|end_header_id|>  \nIf an item is Bad or Very Bad, we don\'t want to sell it anymore.\nHere <classify>"Good"</classify>:"Good"
"""
prompt_2 = prompt_builder(system_prompt=system_prompt_template_2, prompt_data=template_2)
chain_2 = LLMChain(llm=llama_3, prompt=prompt_2, output_key="Review_status_short")

system_prompt_template_3 = "You are a online store sales assistant. Given the item is claissified as Good or Bad, decide if we will keep selling them as [Yes, Neutral, No]."

template_3 = """
If an item is Bad or Very Bad, we don't want to sell it anymore.
Here are some examples:
1. Given the item is claissified as Very Bad, output: "No".
2. Given the item is claissified as Bad, output: "No".
3. Given the item is claissified as Fair, output: "Neutral".
4. Given the item is claissified as Good, output: "Yes".
Now given the item is claissified as {Review_status_short}, output: 
"""

prompt_3 = prompt_builder(system_prompt=system_prompt_template_3, prompt_data=template_3)
chain_3 = LLMChain(llm=llama_3, prompt=prompt_3, output_key="is_keeping")

<class 'langchain_core.prompts.prompt.PromptTemplate'>


/var/folders/v5/3nrys0b97rg31_kvjlgmc7w40000gn/T/ipykernel_966/1516905712.py:15: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain_1 = LLMChain(llm=llama_3, prompt=prompt_1, output_key="Review_status")


In [7]:
# Use SequentialChain to chain the two LLM chains
overall_sequential_chain = SequentialChain(chains=[chain_1, chain_2, chain_3],
                                           input_variables=["discount_percentage", "rating"],
                                           output_variables=["Review_status","Review_status_short", "is_keeping"],
                                           verbose=True)
# sequential_chain = SequentialChain(chains=[chain_1],input_variables=["discount_percentage", "rating"], verbose=True)

my_discount = list(discount_percentage[:15])
my_rating = list(rating[:15]) 
res2 = overall_sequential_chain({"discount_percentage":my_discount[4], "rating":my_rating[4]})
"""
# Input text to be passed to the first chain
for each_discount_percentage, each_rating in zip(my_discount, my_rating):
    # Run the sequential chain
    final_output = sequential_chain.run({"discount_percentage": each_discount_percentage, "rating":each_rating})
    print("Final output from the sequential chain:", final_output)
"""



> Entering new SequentialChain chain...


/var/folders/v5/3nrys0b97rg31_kvjlgmc7w40000gn/T/ipykernel_966/2078259296.py:10: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  res2 = overall_sequential_chain({"discount_percentage":my_discount[4], "rating":my_rating[4]})



> Finished chain.


'\n# Input text to be passed to the first chain\nfor each_discount_percentage, each_rating in zip(my_discount, my_rating):\n    # Run the sequential chain\n    final_output = sequential_chain.run({"discount_percentage": each_discount_percentage, "rating":each_rating})\n    print("Final output from the sequential chain:", final_output)\n'

In [8]:
res2

{'discount_percentage': '61%',
 'rating': '4.2',
 'Review_status': '<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are a online store assistant. Classifying if an item is [Good, Fair, Bad, Very Bad] in <classify></classify> tag. <|eot_id|>\n<|start_header_id|>user<|end_header_id|>  \nIf an item get bad review on a discount, it\'s a bad item.\nHere are some examples:\n1. discount percentage: 62% and rating: 2.2 is "Very Bad".\n2. discount percentage: 30% and rating: 2.2 is "Bad".\n3. discount percentage: 30% and rating 4 is "Fair".\n4. discount percentage: 10% and rating: 4.5 is "Good".\nNow given discount percentage: 61% and rating: 4.2, give only a 1 word review in [Good, Fair, Bad, Very Bad].\n <|eot_id|>\n<|start_header_id|>assistant<|end_header_id|>  <classify>Bad</classify>',
 'Review_status_short': '<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are a helpful text assistant. Get the value within the <classify></classify> tags and output <|eot_id|